In [2]:
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Load the pre-trained EfficientNetB0 model without the top layer
base_model = EfficientNetB7(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)  # Adjust according to your dataset

# # Unfreeze some top layers of the base model
# base_model.trainable = True
# fine_tune_at = 100  # This is an example value; adjust based on your model's architecture
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False
    
# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
initial_learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=initial_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary to inspect its structure and number of parameters
model.summary()

# Set up ImageDataGenerator for training and validation data
batch_size = 32
desired_size = (224, 224)
train_datagen = ImageDataGenerator(
     rescale=1./255,
    # rotation_range=10,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest',
    validation_split=0.2
)

# Adjust the directory paths to the preprocessed dataset
train_generator = train_datagen.flow_from_directory(
    directory='dataset',
    target_size=desired_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    directory='dataset',
    target_size=desired_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Define callbacks for early stopping, model checkpointing, and reducing learning rate on plateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True),
    ModelCheckpoint('best_model_efficientnetb7.keras', monitor='val_loss', save_best_only=True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001, verbose=1)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Save the trained model
model.save('efficientnetb7_finetuned_model.keras')




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]'] 

In [4]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model_sofar.keras', monitor='val_loss', save_best_only=True, mode='min')
model_checkpoint = ModelCheckpoint('best_model_sofar_efficientnetb7_1st_trial.keras', monitor='val_loss', save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001, verbose=1)  # Adjust these parameters
# Unfreeze some top layers of the base model
base_model.trainable = True
fine_tune_at = 100  # This is an example value; adjust based on your model's architecture
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),  # Adjust learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(
    train_generator,
    epochs=10,  # Adjust based on your needs
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

model.save('efficientnetb7_finetuned_1st.keras')


Epoch 1/10
227/227 [==============================] - 1736s 7s/step - loss: 0.3621 - accuracy: 0.8454 - val_loss: 0.5310 - val_accuracy: 0.7742 - lr: 1.0000e-05
Epoch 2/10
227/227 [==============================] - 1640s 7s/step - loss: 0.3217 - accuracy: 0.8646 - val_loss: 0.5198 - val_accuracy: 0.7841 - lr: 1.0000e-05
Epoch 3/10
227/227 [==============================] - 1806s 8s/step - loss: 0.2929 - accuracy: 0.8765 - val_loss: 0.5184 - val_accuracy: 0.7869 - lr: 1.0000e-05
Epoch 4/10
227/227 [==============================] - 1743s 8s/step - loss: 0.2707 - accuracy: 0.8866 - val_loss: 0.5281 - val_accuracy: 0.7957 - lr: 1.0000e-05
Epoch 5/10
227/227 [==============================] - 1678s 7s/step - loss: 0.2500 - accuracy: 0.9017 - val_loss: 0.5177 - val_accuracy: 0.8128 - lr: 1.0000e-05
Epoch 6/10
227/227 [==============================] - 1662s 7s/step - loss: 0.2113 - accuracy: 0.9140 - val_loss: 0.5515 - val_accuracy: 0.8045 - lr: 1.0000e-05
Epoch 7/10
227/227 [==============

In [5]:
from tensorflow.keras.models import load_model

# Assuming your model is saved under 'best_model.keras'
model = load_model('best_model_sofar_efficientnetb7_1st_trial.keras')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32  # Adjust to your preference
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    directory='test_dataset_processed',
    target_size=(224, 224),  # Ensure this matches your model's expected input
    batch_size=batch_size,
    class_mode='categorical',  # or 'binary' for binary classification
    shuffle=False  # Important for confusion matrix
)


Found 1090 images belonging to 2 classes.


In [7]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


35/35 [==============================] - 70s 2s/step - loss: 0.4742 - accuracy: 0.8092
Test Loss: 0.4742313623428345
Test Accuracy: 0.8091742992401123
